In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import json
import requests
import os
import matplotlib.pyplot as plt
import cufflinks as cf
cf.set_config_file(world_readable=True,theme='pearl')
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline
years = [2015,2016,2017,2018,2019]
df={}

for year in years:
    url = f"../UWA-Project/Resources/{year}.csv"
    pd.set_option("display.max_rows",159)
    df[year] = pd.read_csv(url)
    


In [2]:
df_2015,df_2016,df_2017,df_2018,df_2019 = df[2015],df[2016],df[2017],df[2018],df[2019]

In [3]:
for year in years:
    print(f"""{year} \n{df[year].isna().sum()}
----------""")

2015 
Country                          0
Region                           0
Happiness Rank                   0
Happiness Score                  0
Standard Error                   0
Economy (GDP per Capita)         0
Family                           0
Health (Life Expectancy)         0
Freedom                          0
Trust (Government Corruption)    0
Generosity                       0
Dystopia Residual                0
dtype: int64
----------
2016 
Country                          0
Region                           0
Happiness Rank                   0
Happiness Score                  0
Lower Confidence Interval        0
Upper Confidence Interval        0
Economy (GDP per Capita)         0
Family                           0
Health (Life Expectancy)         0
Freedom                          0
Trust (Government Corruption)    0
Generosity                       0
Dystopia Residual                0
dtype: int64
----------
2017 
Country                          0
Happiness.Rank          

In [4]:
df_2018.fillna(0,inplace = True)
df_2018.isna().sum()

Overall rank                    0
Country or region               0
Score                           0
GDP per capita                  0
Social support                  0
Healthy life expectancy         0
Freedom to make life choices    0
Generosity                      0
Perceptions of corruption       0
dtype: int64

In [5]:
for year in years:
    print(f"""{year} \n{df[year].shape}
----------""")

2015 
(158, 12)
----------
2016 
(157, 13)
----------
2017 
(155, 12)
----------
2018 
(156, 9)
----------
2019 
(156, 9)
----------


In [6]:
# get column values
for year in years:
    print(f"""{year} \n{df[year].columns}
----------""")

2015 
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')
----------
2016 
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')
----------
2017 
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')
----------
2018 
Index(['Overall rank', 'Country or region', 'Score', 'GDP per capita',
      

In [7]:
# replacing spaces with underscore
for year in years:
    df[year].columns = [x.replace(" ","_") for x in df[year].columns.values]
    print(f"""{year} \n{df[year].columns}
----------""")



2015 
Index(['Country', 'Region', 'Happiness_Rank', 'Happiness_Score',
       'Standard_Error', 'Economy_(GDP_per_Capita)', 'Family',
       'Health_(Life_Expectancy)', 'Freedom', 'Trust_(Government_Corruption)',
       'Generosity', 'Dystopia_Residual'],
      dtype='object')
----------
2016 
Index(['Country', 'Region', 'Happiness_Rank', 'Happiness_Score',
       'Lower_Confidence_Interval', 'Upper_Confidence_Interval',
       'Economy_(GDP_per_Capita)', 'Family', 'Health_(Life_Expectancy)',
       'Freedom', 'Trust_(Government_Corruption)', 'Generosity',
       'Dystopia_Residual'],
      dtype='object')
----------
2017 
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')
----------
2018 
Index(['Overall_rank', 'Country_or_region', 'Score', 'GDP_per_capita',
      

In [18]:

for year in years:
    df[year].columns = [x.replace(" ","_") for x in df[year].columns.values]
    print(f"""{year} \n{df[year].columns}
----------""")

2015 
Index(['Country', 'Region', 'Happiness_Rank', 'Happiness_Score',
       'Standard_Error', 'Economy_(GDP_per_Capita)', 'Family',
       'Health_(Life_Expectancy)', 'Freedom', 'Trust_(Government_Corruption)',
       'Generosity', 'Dystopia_Residual'],
      dtype='object')
----------
2016 
Index(['Country', 'Region', 'Happiness_Rank', 'Happiness_Score',
       'Upper_Confidence_Interval', 'Economy_(GDP_per_Capita)', 'Family',
       'Health_(Life_Expectancy)', 'Freedom', 'Trust_(Government_Corruption)',
       'Generosity', 'Dystopia_Residual'],
      dtype='object')
----------
2017 
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')
----------
2018 
Index(['Overall_rank', 'Country_or_region', 'Score', 'GDP_per_capita',
       'Social_support', 'Healthy_life_exp

In [8]:
# standard_columns = ['Country']
# def check_columns(rows):
#     if rows not in  standard_columns:
#         del rows

# for year in years:
#     df[year].columns = [del(x) for x in apply(check_columns)]
#     print(f"""{year} \n{df[year].columns}
#         ----------""")
    

In [9]:
df_2015,df_2016,df_2017,df_2018,df_2019 

(                      Country                           Region  \
 0                 Switzerland                   Western Europe   
 1                     Iceland                   Western Europe   
 2                     Denmark                   Western Europe   
 3                      Norway                   Western Europe   
 4                      Canada                    North America   
 5                     Finland                   Western Europe   
 6                 Netherlands                   Western Europe   
 7                      Sweden                   Western Europe   
 8                 New Zealand        Australia and New Zealand   
 9                   Australia        Australia and New Zealand   
 10                     Israel  Middle East and Northern Africa   
 11                 Costa Rica      Latin America and Caribbean   
 12                    Austria                   Western Europe   
 13                     Mexico      Latin America and Caribbea

In [10]:
df[2015].describe()

,Happiness_Rank,Happiness_Score,Standard_Error,Economy_(GDP_per_Capita),Family,Health_(Life_Expectancy),Freedom,Trust_(Government_Corruption),Generosity,Dystopia_Residual
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,79.493671,5.375734,0.047885,0.846137,0.991046,0.630259,0.428615,0.143422,0.237296,2.098977
std,45.754363,1.145010,0.017146,0.403121,0.272369,0.247078,0.150693,0.120034,0.126685,0.553550
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580
25%,40.250000,4.526000,0.037268,0.545808,0.856823,0.439185,0.328330,0.061675,0.150553,1.759410
50%,79.500000,5.232500,0.043940,0.910245,1.029510,0.696705,0.435515,0.107220,0.216130,2.095415
75%,118.750000,6.243750,0.052300,1.158448,1.214405,0.811013,0.549092,0.180255,0.309883,2.462415
max,158.000000,7.587000,0.136930,1.690420,1.402230,1.025250,0.669730,0.551910,0.795880,3.602140


In [11]:
columns = ['Country','Region','Happiness Rank','Happiness Score','Economy (GDP per Capita),Family,Health (Life Expectancy)','Freedom','Trust (Government Corruption)','Generosity','Dystopia Residual']
df[2016].columns

Index(['Country', 'Region', 'Happiness_Rank', 'Happiness_Score',
       'Lower_Confidence_Interval', 'Upper_Confidence_Interval',
       'Economy_(GDP_per_Capita)', 'Family', 'Health_(Life_Expectancy)',
       'Freedom', 'Trust_(Government_Corruption)', 'Generosity',
       'Dystopia_Residual'],
      dtype='object')

In [23]:
col = ['Lower_Confidence_Interval','Upper_Confidence_Interval','Region','Whisker.high','Whisker.low','Standard_Error']
for year in years:
    for value in df[year].columns:
          if value in col:
            del df[year][value]
#         del df[year]['Lower_Confidence_Interval']
df[2017]

,Country,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182
5,Netherlands,6,7.377,1.503945,1.428939,0.810696,0.585384,0.470490,0.282662,2.294804
6,Canada,7,7.316,1.479204,1.481349,0.834558,0.611101,0.435540,0.287372,2.187264
7,New Zealand,8,7.314,1.405706,1.548195,0.816760,0.614062,0.500005,0.382817,2.046456
8,Sweden,9,7.284,1.494387,1.478162,0.830875,0.612924,0.385399,0.384399,2.097538
9,Australia,10,7.284,1.484415,1.510042,0.843887,0.601607,0.477699,0.301184,2.065211


In [24]:
for year in years:
    print(f"""{year} \n{df[year].shape}
----------""")

2015 
(158, 10)
----------
2016 
(157, 10)
----------
2017 
(155, 10)
----------
2018 
(156, 9)
----------
2019 
(156, 9)
----------


In [ ]:
df_2018['Dystop']

In [ ]:
plt.figure(figsize=(20,12))
sns.heatmap(df[2016].corr(), annot=True)
plt.show()

In [ ]:
sns.pairplot(df[2016])

In [ ]:
cf.help('bar')